In this notebook we explore the noise models provided by Cirq. Beforehand let's see that we can actually wrap this in tfq.Expectation layer.

In [80]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt

n_qubits = 4
qubits = cirq.GridQubit.rect(1, n_qubits)
g=1
J=1.21
observable = [-float(0.5*g)*cirq.Z.on(q) for q in qubits] 
for q in range(len(qubits)):
    observable.append(-float(0.5*J)*cirq.X.on(qubits[q])*cirq.X.on(qubits[(q+1)%len(qubits)]))
gate=[cirq.rz, cirq.rx, cirq.rz]

circuit = []
symbols=[]
for k in range(len(qubits)-1):
    for j in range(3):
        symbol="th_"+str(k)+"_"+str(j)
        symbols.append(symbol)
        circuit.append(gate[j](sympy.Symbol(symbol)).on(qubits[k]))
        #circuit.append(cirq.depolarize(p=0.01).on(qubits[k]))
for k in range(len(qubits)):
    circuit.append(cirq.CNOT(qubits[k], qubits[(k+1)%len(qubits)]))
circuit = cirq.Circuit(circuit)

In [81]:
circuit

(0, 0): ───Rz(th_0_0)───Rx(th_0_1)───Rz(th_0_2)───@───────────X───
                                                  │           │
(0, 1): ───Rz(th_1_0)───Rx(th_1_1)───Rz(th_1_2)───X───@───────┼───
                                                      │       │
(0, 2): ───Rz(th_2_0)───Rx(th_2_1)───Rz(th_2_2)───────X───@───┼───
                                                          │   │
(0, 3): ──────────────────────────────────────────────────X───@───

cirq.ConstantQubitNoiseModel(cirq.depolarize(p=0.01))

In [82]:
cc=circuit.with_noise(cirq.ConstantQubitNoiseModel.from_noise_model_like(cirq.depolarize(p=0.01)))
cc

(0, 0): ───Rz(th_0_0)───D(0.01)[<virtual>]───Rx(th_0_1)───D(0.01)[<virtual>]───Rz(th_0_2)───D(0.01)[<virtual>]───@───D(0.01)[<virtual>]───────D(0.01)[<virtual>]───────D(0.01)[<virtual>]───X───D(0.01)[<virtual>]───
                                                                                                                 │                                                                          │
(0, 1): ───Rz(th_1_0)───D(0.01)[<virtual>]───Rx(th_1_1)───D(0.01)[<virtual>]───Rz(th_1_2)───D(0.01)[<virtual>]───X───D(0.01)[<virtual>]───@───D(0.01)[<virtual>]───────D(0.01)[<virtual>]───┼───D(0.01)[<virtual>]───
                                                                                                                                          │                                                 │
(0, 2): ───Rz(th_2_0)───D(0.01)[<virtual>]───Rx(th_2_1)───D(0.01)[<virtual>]───Rz(th_2_2)───D(0.01)[<virtual>]───────D(0.01)[<virtual>]───X───D(0.01)[<virtual>]───@───D(0.01)[<virtual>]───┼───D(0.01)[<virtual>]───
                                                                                                                                                                   │                        │
(0, 3): ────────────────D(0.01)[<virtual>]────────────────D(0.01)[<virtual>]────────────────D(0.01)[<virtual>]───────D(0.01)[<virtual>]───────D(0.01)[<virtual>]───X───D(0.01)[<virtual>]───@───D(0.01)[<virtual>]───

In [ ]:
cc=circuit.with_noise(cirq.ConstantQubitNoiseModel.from_noise_model_like(cirq.depolarize(p=0.01)))
cc

In [46]:
simulator = cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.01))

In [54]:
res = simulator.simulate(circuit, param_resolver={s:k for s,k in zip(symbols,np.random.randn(len(symbols)))})

In [60]:
res.final_density_matrix

array([[ 5.62487245e-01+1.45213823e-08j,  7.36966683e-03+1.19510284e-02j,
         1.80307209e-01+1.51208133e-01j, -8.50309618e-04+5.81205776e-03j,
         1.42710358e-01-6.53418899e-02j,  3.25808208e-03+2.17602565e-03j,
         6.33114651e-02+1.74178891e-02j,  4.59428702e-04+1.57337147e-03j,
        -2.61589549e-02-2.97069490e-01j,  5.96902799e-03-4.44797520e-03j,
         7.14730322e-02-1.02258690e-01j,  3.10909795e-03+1.78783841e-04j,
        -4.11462262e-02-7.23316073e-02j,  9.97716561e-04-1.82190666e-03j,
         6.25465158e-03-3.42470892e-02j,  8.09587247e-04-3.15811660e-04j],
       [ 7.36966636e-03-1.19510274e-02j,  1.16887707e-02-5.82655035e-10j,
         5.57505246e-03-1.84982480e-03j,  3.74687579e-03+3.14218132e-03j,
         4.81477211e-04-3.88823426e-03j,  2.96559324e-03-1.35783781e-03j,
         1.19957572e-03-1.11695589e-03j,  1.31564494e-03+3.61952523e-04j,
        -6.65449258e-03-3.33639001e-03j, -5.43596165e-04-6.17325539e-03j,
        -1.23623048e-03-2.85835448e-0

In [ ]:
tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(self.noise_level)))(
        tfqcircuit,
        operators=tfq.convert_to_tensor([self.observable]))

In [ ]:
#https://cirq.readthedocs.io/en/stable/_modules/cirq/ops/common_channels.html#depolarize


from typing import (Any, Dict, Iterable, Optional, Sequence, Tuple, Union,
                    TYPE_CHECKING)

import numpy as np

from cirq import protocols, value
from cirq.ops import (raw_types, common_gates, pauli_gates, gate_features,
                      identity)

if TYPE_CHECKING:
    import cirq

@value.value_equality
class AsymmetricDepolarizingChannel(gate_features.SingleQubitGate):

    def __init__(self, p_x: float, p_y: float, p_z: float) -> None:

        self._p_x = value.validate_probability(p_x, 'p_x')
        self._p_y = value.validate_probability(p_y, 'p_y')
        self._p_z = value.validate_probability(p_z, 'p_z')
        self._p_i = 1 - value.validate_probability(p_x + p_y + p_z,
                                                   'p_x + p_y + p_z')


    def _mixture_(self) -> Sequence[Tuple[float, np.ndarray]]:
        return ((self._p_i, protocols.unitary(identity.I)),
                (self._p_x, protocols.unitary(pauli_gates.X)),
                (self._p_y, protocols.unitary(pauli_gates.Y)),
                (self._p_z, protocols.unitary(pauli_gates.Z)))

    def _has_mixture_(self) -> bool:
        return True

    def _value_equality_values_(self):
        return self._p_x, self._p_y, self._p_z

    def __repr__(self) -> str:
        return 'cirq.asymmetric_depolarize(p_x={!r},p_y={!r},p_z={!r})'.format(
            self._p_x, self._p_y, self._p_z
        )

    def __str__(self) -> str:
        return 'asymmetric_depolarize(p_x={!r},p_y={!r},p_z={!r})'.format(
            self._p_x, self._p_y, self._p_z
        )

    def _circuit_diagram_info_(self,
                               args: 'protocols.CircuitDiagramInfoArgs') -> str:
        if args.precision is not None:
            f = '{:.' + str(args.precision) + 'g}'
            return 'A({},{},{})'.format(f, f, f).format(self._p_x, self._p_y,
                                                        self._p_z)
        return 'A({!r},{!r},{!r})'.format(self._p_x, self._p_y, self._p_z)

    @property
    def p_x(self) -> float:
        """The probability that a Pauli X and no other gate occurs."""
        return self._p_x

    @property
    def p_y(self) -> float:
        """The probability that a Pauli Y and no other gate occurs."""
        return self._p_y

    @property
    def p_z(self) -> float:
        """The probability that a Pauli Z and no other gate occurs."""
        return self._p_z

    def _json_dict_(self) -> Dict[str, Any]:
        return protocols.obj_to_dict_helper(self, ['p_x', 'p_y', 'p_z'])